##### Copyright 2019 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.


# Creación de un modelo lineal con estimadores

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/tutorials/estimator/linear"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver en TensorFlow.org</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/es-419/tutorials/estimator/linear.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Ejecutar en Google Colab</a></td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/es-419/tutorials/estimator/linear.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fuente en GitHub</a> </td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/es-419/tutorials/estimator/linear.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Descargar notebook</a> </td>
</table>

> Advertencia: Los estimadores no se recomiendan para código nuevo. Los estimadores ejecutan el código de estilo `v1.Session` que es más difícil de escribir correctamente y que puede tener un comportamiento inesperado; particularmente, cuando se combina con código TF 2. Los estimadores están incluidos dentro de nuestras [garantías de compatibilidad](https://tensorflow.org/guide/versions), pero no se les harán correcciones a menos que se trate de vulneraciones a la seguridad. Para más detalles, consulte la [Guía de migración](https://tensorflow.org/guide/migrate).

## Descripción general

Esta guía completa, paso a paso, entrena al modelo de regresión logística con la API `tf.estimator`. Por lo general, el modelo se usa como base de referencia para otros algoritmos más complejos.

Nota: hay un ejemplo de regresión logística de Keras [disponible](https://tensorflow.org/guide/migrate/tutorials/keras/regression) y se recomienda en este tutorial.


## Preparar

In [ ]:
!pip install sklearn


In [ ]:
import os
import sys

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import clear_output
from six.moves import urllib

## Carga del conjunto de datos (dataset) del Titanic

Usará el conjunto de datos del Titanic con el objetivo (admitamos que, algo morboso) de predecir la supervivencia de los pasajeros según características como el género, la edad, la clase social, etc.

In [ ]:
import tensorflow.compat.v2.feature_column as fc

import tensorflow as tf

In [ ]:
# Load dataset.
dftrain = pd.read_csv('https://storage.googleapis.com/tf-datasets/titanic/train.csv')
dfeval = pd.read_csv('https://storage.googleapis.com/tf-datasets/titanic/eval.csv')
y_train = dftrain.pop('survived')
y_eval = dfeval.pop('survived')

## Exploración de los datos

El conjunto de datos contiene las siguientes características

In [ ]:
dftrain.head()

In [ ]:
dftrain.describe()

Hay 627 y 264 ejemplos en los conjuntos de entrenamiento y evaluación respectivamente.

In [ ]:
dftrain.shape[0], dfeval.shape[0]

La mayoría de los pasajeros tienen más de 20 (entre 20 y 29) o más de 30 (entre 30 a 39) años.

In [ ]:
dftrain.age.hist(bins=20)

La cantidad de pasajeros masculinos a bordo es, aproximadamente, el doble que la de pasajeros femeninos.

In [ ]:
dftrain.sex.value_counts().plot(kind='barh')

La mayoría de los pasajeros son de "segunda" clase.

In [ ]:
dftrain['class'].value_counts().plot(kind='barh')

Las mujeres tiene muchas más posibilidades de sobrevivir que los hombres. Claramente, se trata de una característica predictiva del modelo.

In [ ]:
pd.concat([dftrain, y_train], axis=1).groupby('sex').survived.mean().plot(kind='barh').set_xlabel('% survive')

## Ingeniería de características para el modelo

> Advertencia: El módulo tf.feature_columns descripto en este tutorial no se recomienda para código nuevo. Las capas de procesamiento Keras incluyen esta funcionalidad. Para acceder a las instrucciones de migración, consulte la [guía de características de migración](https://www.tensorflow.org/guide/migrate/migrating_feature_columns). El módulo tf.feature_columns fue diseñado para ser usado con estimadores TF1. Está incluido dentro de nuestras [garantías de compatibilidad](https://tensorflow.org/guide/versions), pero no se le harán correcciones a menos que se trate de vulneraciones a la seguridad.

Para los estimadores se usa un sistema denominado [columnas de atributos](https://www.tensorflow.org/tutorials/structured_data/feature_columns) (feature columns) a fin de describir cómo debe interpretar el modelo cada uno de los atributos de entrada en bruto. Un estimador espera un vector de entradas numéricas y las *columnas de atributos* describen cómo el modelo debería convertir cada atributo.

Seleccionar y elaborar el conjunto correcto de columnas de atributos es la clave para entrenar un modelo efectivo. Una columna de atributos puede ser una lista de entradas en bruto en los atributos originales `dict` (una *columna de atributos base*), o cualquier columna nueva creada con las transformaciones definidas sobre una o varias columnas base (una *columna de atributos derivados*).

El estimador lineal usa tanto los atributos numéricos como los categóricos. Las columnas de atributos funcionan con todos los estimadores de TensorFlow y su propósito es el de definir los atributos usados para crear el modelo. Además, habilitan algunas capacidades de ingeniería de características como la codificación one-hot, la normalización y los datos agrupados (bucketización).

### Columnas de atributos base

In [ ]:
CATEGORICAL_COLUMNS = ['sex', 'n_siblings_spouses', 'parch', 'class', 'deck',
                       'embark_town', 'alone']
NUMERIC_COLUMNS = ['age', 'fare']

feature_columns = []
for feature_name in CATEGORICAL_COLUMNS:
  vocabulary = dftrain[feature_name].unique()
  feature_columns.append(tf.feature_column.categorical_column_with_vocabulary_list(feature_name, vocabulary))

for feature_name in NUMERIC_COLUMNS:
  feature_columns.append(tf.feature_column.numeric_column(feature_name, dtype=tf.float32))

El código `input_function` especifica cómo se convierten los datos a una `tf.data.Dataset` que alimenta a la canalización de entrada en forma de transmisión (streaming). `tf.data.Dataset` puede recibir muchas fuentes como un marco de datos (dataframe), un archivo con formato CSV y muchas más.

In [ ]:
def make_input_fn(data_df, label_df, num_epochs=10, shuffle=True, batch_size=32):
  def input_function():
    ds = tf.data.Dataset.from_tensor_slices((dict(data_df), label_df))
    if shuffle:
      ds = ds.shuffle(1000)
    ds = ds.batch(batch_size).repeat(num_epochs)
    return ds
  return input_function

train_input_fn = make_input_fn(dftrain, y_train)
eval_input_fn = make_input_fn(dfeval, y_eval, num_epochs=1, shuffle=False)

Se puede inspeccionar el conjunto de datos:

In [ ]:
ds = make_input_fn(dftrain, y_train, batch_size=10)()
for feature_batch, label_batch in ds.take(1):
  print('Some feature keys:', list(feature_batch.keys()))
  print()
  print('A batch of class:', feature_batch['class'].numpy())
  print()
  print('A batch of Labels:', label_batch.numpy())

También se pueden inspeccionar los resultados de una columna de atributos específicos con la capa `tf.keras.layers.DenseFeatures`:

In [ ]:
age_column = feature_columns[7]
tf.keras.layers.DenseFeatures([age_column])(feature_batch).numpy()

`DenseFeatures` solamente admite tensores densos, para inspeccionar una columna categórica deberá transformarla primero en columna indicadora

In [ ]:
gender_column = feature_columns[0]
tf.keras.layers.DenseFeatures([tf.feature_column.indicator_column(gender_column)])(feature_batch).numpy()

Después de agregar todos los atributos base al modelo, entrenemos al modelo. El entrenamiento de un modelo es, sencillamente, un solo comando con la API the `tf.estimator` API:

In [ ]:
linear_est = tf.estimator.LinearClassifier(feature_columns=feature_columns)
linear_est.train(train_input_fn)
result = linear_est.evaluate(eval_input_fn)

clear_output()
print(result)

### Columnas de atributos derivados

Ahora, ha llegado a una exactitud del 75%. Con las columnas de atributos base por separado puede no ser suficiente para explicar los datos. Por ejemplo, la correlación entre la edad y la etiqueta pueden ser diferentes cuando los géneros son diferentes. Por lo tanto, si solamente se entrena un solo peso del modelo para `gender="Male"` (género: hombre) y `gender="Female"` (género: mujer), no se capturarán todas las combinaciones de edad y género (p. ej., distinguir entre `gender="Male"` (género: hombre) Y `age="30"` (edad: 30) Y `gender="Male"` (género: hombre) Y `age="40"`) (edad: 40).

Para aprender las diferencias entre distintas combinaciones de atributos, puede agregar *columnas de atributos cruzados* al modelo (también se puede agrupar (bucketizar) la columna de edades antes de la columna cruzada):

In [ ]:
age_x_gender = tf.feature_column.crossed_column(['age', 'sex'], hash_bucket_size=100)

Después de agregar el atributo de combinación al modelo, volvamos a entrenar el modelo:

In [ ]:
derived_feature_columns = [age_x_gender]
linear_est = tf.estimator.LinearClassifier(feature_columns=feature_columns+derived_feature_columns)
linear_est.train(train_input_fn)
result = linear_est.evaluate(eval_input_fn)

clear_output()
print(result)

Ahora logra una exactitud del 77.6 %, algo un poco mejor que con el entrenamiento solamente con atributos base. Se puede intentar usar más atributos y transformaciones para ver si es posible mejorar.

Este es el momento en que se puede usar el modelo entrenado para hacer predicciones sobre uno de los pasajeros del conjunto en evaluación. Los modelos TensorFlow están optimizados para hacer predicciones de un lote, una recopilación o de varios ejemplos a la vez. Antes, `eval_input_fn` se definió usando el conjunto completo en evaluación.

In [ ]:
pred_dicts = list(linear_est.predict(eval_input_fn))
probs = pd.Series([pred['probabilities'][1] for pred in pred_dicts])

probs.plot(kind='hist', bins=20, title='predicted probabilities')

Finalmente, observe la Característica Operativa del Receptor (ROC) de resultados, que nos dará una mejor idea del equilibrio (tradeoff) entre la tasa de verdaderos positivos y la tasa falsos positivos.

In [ ]:
from sklearn.metrics import roc_curve
from matplotlib import pyplot as plt

fpr, tpr, _ = roc_curve(y_eval, probs)
plt.plot(fpr, tpr)
plt.title('ROC curve')
plt.xlabel('false positive rate')
plt.ylabel('true positive rate')
plt.xlim(0,)
plt.ylim(0,)